In [25]:
import numpy as np
import pandas as pd

In [26]:
from scipy.stats import mstats

In [64]:
# Read the data

df = pd.read_excel('./Suppliers Quality Analaysis.xlsx', sheet_name=None)

In [65]:
# Read all sheets data

vendor = df['Vendor']
plant = df['Plant']
defected_items = df['Defected Items']
material_type = df['Material Type']
defects = df['Defects']
defect_type = df['Defect Type']
category = df['Category']

**Remove Duplicates From Vendor**

In [66]:
# Step 1: Identify vendors that have duplicates in the 'Vendor' column
duplicate_vendors = vendor[vendor.duplicated(subset='Vendor', keep=False)]

# Step 2: Display all rows for vendors with duplicates
duplicate_vendors


,Vendor,Vendor ID
79,roundphase,80
112,roundphase,113
124,Quotefix,125
143,Quotefix,144


In [67]:
first_occurrences = duplicate_vendors.drop_duplicates(subset='Vendor', keep='first')

first_occurrences

,Vendor,Vendor ID
79,roundphase,80
124,Quotefix,125


In [68]:
duplicate_dict = {}


for vendor_name in first_occurrences['Vendor']:
    # Get all duplicates for this vendor excluding the first occurrence
    duplicates = duplicate_vendors[(duplicate_vendors['Vendor'] == vendor_name) & 
                                    (duplicate_vendors['Vendor ID'] != first_occurrences[first_occurrences['Vendor'] == vendor_name]['Vendor ID'].values[0])]
    
    # Add the duplicates to the dictionary
    duplicate_dict[vendor_name] = duplicates['Vendor ID'].tolist()

duplicate_dict

{'roundphase': [113], 'Quotefix': [144]}

In [69]:
# Step 1: Identify the first occurrence of each duplicated vendor
first_occurrences = vendor.drop_duplicates(subset='Vendor', keep='first')

# Step 2: Create a mapping of all vendor names to their first occurrence Vendor ID
vendor_mapping = dict(zip(vendor['Vendor'], vendor['Vendor ID']))

# Step 3: Create a reverse mapping for duplicates (replace duplicate Vendor IDs with first occurrence)
for vendor_name in first_occurrences['Vendor']:
    first_id = first_occurrences[first_occurrences['Vendor'] == vendor_name]['Vendor ID'].values[0]
    duplicate_ids = vendor[vendor['Vendor'] == vendor_name]['Vendor ID'].tolist()
    
    # Update the mapping for all duplicate IDs
    for dup_id in duplicate_ids:
        vendor_mapping[dup_id] = first_id

# Step 4: Apply this mapping to update the 'Vendor ID' in defected_items DataFrame
defected_items['Vendor ID'] = defected_items['Vendor ID'].replace(vendor_mapping)


In [70]:
defected_items[defected_items['Vendor ID'] == 125]

,Date,Sub Category ID,Plant ID,Vendor ID,Material ID,Defect Type ID,Material Type ID,Defect ID,Defect Qty,Downtime min
2084,2014-06-09,3,4,125,1475,1,9,172,8544,0.0
2767,2014-03-12,3,4,125,735,1,9,129,0,0.0
3850,2013-11-11,3,4,125,735,1,9,78,0,0.0
4079,2013-10-15,3,4,125,609,1,9,142,0,0.0
5569,2013-03-11,3,4,125,735,1,9,78,0,0.0
5798,2013-02-15,3,4,125,609,1,9,142,0,0.0


In [71]:
# drop the duplicates without the first occurrence from vendor

vendor = vendor.drop_duplicates(subset='Vendor', keep='first')


In [72]:
vendor[vendor['Vendor ID'] == 113]

,Vendor,Vendor ID


**Remove Duplicates from defects**

In [73]:
# Step 1: Identify vendors that have duplicates in the 'Vendor' column
duplicate_defects = defects[defects.duplicated(subset='Defect', keep=False)]

# Step 2: Display all rows for vendors with duplicates
duplicate_defects

,Defect,Defect ID
1,Bad Seams,2
2,Bad Seams,3
3,Bad Seams,4
14,Scrap attached,15
19,Scrap attached,20
23,Too Stiff,24
28,Warped,29
37,Damaged in Transit,38
40,Out of Spec,41
42,Wrong Core,43


In [74]:
first_occurrences_defects = duplicate_defects.drop_duplicates(subset='Defect', keep='first')

first_occurrences_defects

,Defect,Defect ID
1,Bad Seams,2
14,Scrap attached,15
23,Too Stiff,24
28,Warped,29
37,Damaged in Transit,38
40,Out of Spec,41
42,Wrong Core,43
44,Split Seams,45
50,No Adhesive,51
59,Wrong Registration,60


In [75]:
# Step 1: Identify the first occurrence of each duplicated vendor
first_occurrences_defects = defects.drop_duplicates(subset='Defect', keep='first')

# Step 2: Create a mapping of all vendor names to their first occurrence Vendor ID
defects_mapping = dict(zip(defects['Defect'], defects['Defect ID']))

# Step 3: Create a reverse mapping for duplicates (replace duplicate Vendor IDs with first occurrence)
for defect_name in first_occurrences_defects['Defect']:
    first_id = first_occurrences_defects[first_occurrences_defects['Defect'] == defect_name]['Defect ID'].values[0]
    duplicate_ids = defects[defects['Defect'] == defect_name]['Defect ID'].tolist()
    
    # Update the mapping for all duplicate IDs
    for dup_id in duplicate_ids:
        defects_mapping[dup_id] = first_id

# Step 4: Apply this mapping to update the 'Vendor ID' in defected_items DataFrame
defected_items['Defect ID'] = defected_items['Defect ID'].replace(defects_mapping)

In [76]:
defects = defects.drop_duplicates(subset='Defect', keep='first')

In [77]:
defected_items[defected_items['Defect ID'] == 99]

,Date,Sub Category ID,Plant ID,Vendor ID,Material ID,Defect Type ID,Material Type ID,Defect ID,Defect Qty,Downtime min


**Deleted Outlier Record**

In [78]:
# Delete row that have Defect Type ID = 8 in defected_items

defected_items = defected_items[defected_items['Defect Type ID'] != 8]

In [79]:
defected_items[defected_items['Defect ID'] == 19580]

,Date,Sub Category ID,Plant ID,Vendor ID,Material ID,Defect Type ID,Material Type ID,Defect ID,Defect Qty,Downtime min


In [80]:
# seperate the Plant column in plant dataframe to 2 columns by comma , and drop the Plant column

plant[['Plant', 'State']] = plant['Plant'].str.split(',', expand=True)

plant

,Plant,Plant ID,State
0,Grand Rapids,1,MI
1,Milwaukee,2,Wi
2,Springfield,3,IL
3,Chicago,4,IL
4,Indianapolis,5,IN
5,Northbrook,6,IL
6,Detriot,7,MI
7,Gary,8,IN
8,Joliet,9,IL
9,Monon,10,IN


In [81]:
plant['State Name'] = plant['State'].str.strip().str.upper().map({'MI': 'Michigan', 'WI': 'Wisconsin', 'IL': 'Illinois', 'IN': 'Indiana', 'OH': 'Ohio', 'IA': 'Iowa'})

plant

,Plant,Plant ID,State,State Name
0,Grand Rapids,1,MI,Michigan
1,Milwaukee,2,Wi,Wisconsin
2,Springfield,3,IL,Illinois
3,Chicago,4,IL,Illinois
4,Indianapolis,5,IN,Indiana
5,Northbrook,6,IL,Illinois
6,Detriot,7,MI,Michigan
7,Gary,8,IN,Indiana
8,Joliet,9,IL,Illinois
9,Monon,10,IN,Indiana


In [82]:
# add longitude and latitude columns to plant dataframe
# State = State Name => longitude, latitude
# MI = Michigan   => -84.506836 , 44.182205
# WI = Wisconsin  => -89.500000 , 44.500000
# IL = Illinois   => -89.000000 , 40.000000
# IN = Indiana    => -86.126976 , 40.273502
# OH = Ohio       => -82.996216 , 40.367474
# IA = Iowa       => -93.581543 , 42.032974

plant['Longitude'] = plant['State Name'].map({'Michigan': -84.506836, 'Wisconsin': -89.500000, 'Illinois': -89.000000, 'Indiana': -86.126976, 'Ohio': -82.996216, 'Iowa': -93.581543})
plant['Latitude'] = plant['State Name'].map({'Michigan': 44.182205, 'Wisconsin': 44.500000, 'Illinois': 40.000000, 'Indiana': 40.273502, 'Ohio': 40.367474, 'Iowa': 42.032974})

plant

,Plant,Plant ID,State,State Name,Longitude,Latitude
0,Grand Rapids,1,MI,Michigan,-84.506836,44.182205
1,Milwaukee,2,Wi,Wisconsin,-89.500000,44.500000
2,Springfield,3,IL,Illinois,-89.000000,40.000000
3,Chicago,4,IL,Illinois,-89.000000,40.000000
4,Indianapolis,5,IN,Indiana,-86.126976,40.273502
5,Northbrook,6,IL,Illinois,-89.000000,40.000000
6,Detriot,7,MI,Michigan,-84.506836,44.182205
7,Gary,8,IN,Indiana,-86.126976,40.273502
8,Joliet,9,IL,Illinois,-89.000000,40.000000
9,Monon,10,IN,Indiana,-86.126976,40.273502


**Remove Outliers From Defected Items**

In [83]:
defected_items[['Defect Qty', 'Downtime min']].describe()

,Defect Qty,Downtime min
count,6144.000000,6144.000000
mean,9113.179525,22.665690
std,30586.032560,75.376638
min,0.000000,0.000000
25%,6.000000,0.000000
50%,438.000000,0.000000
75%,5127.000000,20.000000
max,487008.000000,999.000000


In [47]:
# Define the lower and upper percentiles for Winsorization
lower_percentile = 0.05  # 5th percentile
upper_percentile = 0.95   # 95th percentile

# Winsorize the specified columns
defected_items['Defect Qty'] = mstats.winsorize(defected_items['Defect Qty'], limits=[lower_percentile, upper_percentile])
defected_items['Downtime min'] = mstats.winsorize(defected_items['Downtime min'], limits=[lower_percentile, upper_percentile])

# Optionally, check the result
defected_items[['Defect Qty', 'Downtime min']].describe()

C:\Program Files\KMSpico\temp\ipykernel_18480\1173237536.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defected_items['Defect Qty'] = mstats.winsorize(defected_items['Defect Qty'], limits=[lower_percentile, upper_percentile])
C:\Program Files\KMSpico\temp\ipykernel_18480\1173237536.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defected_items['Downtime min'] = mstats.winsorize(defected_items['Downtime min'], limits=[lower_percentile, upper_percentile])


,Defect Qty,Downtime min
count,6144.0,6144.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


In [84]:
# Define the lower and upper percentiles for clipping
lower_limit_defect_qty = defected_items['Defect Qty'].quantile(lower_percentile)
upper_limit_defect_qty = defected_items['Defect Qty'].quantile(upper_percentile)

lower_limit_downtime_min = defected_items['Downtime min'].quantile(lower_percentile)
upper_limit_downtime_min = defected_items['Downtime min'].quantile(upper_percentile)

# Clip the values
defected_items['Defect Qty'] = defected_items['Defect Qty'].clip(lower=lower_limit_defect_qty, upper=upper_limit_defect_qty)
defected_items['Downtime min'] = defected_items['Downtime min'].clip(lower=lower_limit_downtime_min, upper=upper_limit_downtime_min)

# Check the result
defected_items[['Defect Qty', 'Downtime min']].describe()


C:\Program Files\KMSpico\temp\ipykernel_18480\2801352225.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defected_items['Defect Qty'] = defected_items['Defect Qty'].clip(lower=lower_limit_defect_qty, upper=upper_limit_defect_qty)
C:\Program Files\KMSpico\temp\ipykernel_18480\2801352225.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defected_items['Downtime min'] = defected_items['Downtime min'].clip(lower=lower_limit_downtime_min, upper=upper_limit_downtime_min)


,Defect Qty,Downtime min
count,6144.000000,6144.000000
mean,6012.776855,14.179850
std,11618.516783,25.335497
min,0.000000,0.000000
25%,6.000000,0.000000
50%,438.000000,0.000000
75%,5127.000000,20.000000
max,42275.000000,90.000000


**Fill the missing value in the Downtime min with mode**

In [85]:
# Calculate the mode
downtime_mode = defected_items['Downtime min'].mode()[0]

# Fill missing values with mode
defected_items['Downtime min'].fillna(downtime_mode, inplace=True)


C:\Program Files\KMSpico\temp\ipykernel_18480\3362259466.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defected_items['Downtime min'].fillna(downtime_mode, inplace=True)


In [86]:
# Optionally, check the result
defected_items['Downtime min'].isnull().sum()

0

**Save The Files**

In [87]:
# save all dataframes to excel file

with pd.ExcelWriter('Suppliers Quality Analysis - Cleaned.xlsx') as writer:
    vendor.to_excel(writer, sheet_name='Vendor', index=False)
    plant.to_excel(writer, sheet_name='Plant', index=False)
    defected_items.to_excel(writer, sheet_name='Defected Items', index=False)
    material_type.to_excel(writer, sheet_name='Material Type', index=False)
    defects.to_excel(writer, sheet_name='Defects', index=False)
    defect_type.to_excel(writer, sheet_name='Defect Type', index=False)
    category.to_excel(writer, sheet_name='Category', index=False)